In [1]:

import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [2]:
!pip install -q langchain



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.6/120.6 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 19.9 MB/s eta 0:00:00


In [3]:
import gc, inspect, json, re
import xml.etree.ElementTree as ET
from functools import partial
from typing import get_type_hints

import transformers
import torch

from langchain.chains.openai_functions import convert_to_openai_function
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from langchain.pydantic_v1 import BaseModel, Field, validator

In [4]:
model_name ="NexaAIDev/Octopus-v4"


In [5]:

def load_model(model_name: str):
    tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

    with torch.device("cuda:0"):
        model = transformers.AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16).eval()

    return tokenizer, model


In [6]:
tokenizer, model = load_model(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/9.25k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.85M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

In [7]:
def delete_model(*args):
    for var in args:
        if var in globals():
            del globals()[var]

    gc.collect()
    torch.cuda.empty_cache()

In [8]:
# Define the generation_config
generation_config = {
    "use_cache": True,
    "do_sample": True,
    "temperature": 0.2,
    "top_p": 1.0,
    "top_k": 0,
    "max_new_tokens": 512,
    "eos_token_id": tokenizer.eos_token_id,
    "pad_token_id": tokenizer.eos_token_id,
}

# Generate tokens for the prompt "Tell me a joke"
prompt = "Tell me a joke"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Ensure model_kwargs is properly defined
model_kwargs = {"generation_config": generation_config}
model_kwargs

# generated_tokens = model.generate(input_ids=inputs["input_ids"], **model_kwargs)
# print("Generated tokens:", generated_tokens)


{'generation_config': {'use_cache': True,
  'do_sample': True,
  'temperature': 0.2,
  'top_p': 1.0,
  'top_k': 0,
  'max_new_tokens': 512,
  'eos_token_id': 32000,
  'pad_token_id': 32000}}

In [9]:
inputs

{'input_ids': tensor([[    1, 24948,   592,   263,  2958,   446]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [10]:
from transformers import GenerationConfig

# Define the generation configuration
generation_config = GenerationConfig(
    pad_token_id=tokenizer.pad_token_id,
    temperature=0.6,
    top_p=0.7,
    top_k=50,
    num_beams=1,
    max_length=1024,
    do_sample=True,
)

# Generate tokens for the prompt "Tell me a joke"
# prompt = "Send me a mail to sawradip0@gmail.com about early leave from radeensust@gmail.com"
prompt="I am sick. What can I do?"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Generate tokens using the model
generation_output = model.generate(
    input_ids=inputs["input_ids"],
    generation_config=generation_config,
    return_dict_in_generate=True,
    output_scores=True,
    do_sample=True,
    max_length=1024,  # Adjusted max_new_tokens to max_length
)

# Extract generated tokens
generated_tokens = generation_output.sequences
print("Generated tokens:", generated_tokens)

Generated tokens: tensor([[    1,   306,   626, 17319, 29889,  1724,   508,   306,   437, 29973,
         32007]], device='cuda:0')


In [11]:
tokenizer.decode(generated_tokens[0])

'<s> I am sick. What can I do?<|end|>'

In [30]:
input_text="Write an email to my friend arik@gmail.com about I couldn't attend his wedding."
nexa_query=f"""Below is the query from the users, please call the correct function and generate parameters
to call the function. \n\nQuery: {input_text} \n\nResponse:"""

In [31]:
input_ids=tokenizer(nexa_query,return_tensors="pt").to(model.device)
input_length=input_ids["input_ids"].shape[1]

In [32]:
outputs=model.generate(
    input_ids=input_ids["input_ids"],
    max_length=1024,
    do_sample=True
)
generated_sequence=outputs[:, input_length:].tolist()
res=tokenizer.decode(generated_sequence[0])

In [33]:
print(res)

<nexa_16> ('Craft a considerate email notifying a friend of the inability to attend their wedding ceremony, expressing regret and best wishes.')<nexa_end> 

Function description: 
def general_gpt(query):
    """
    A general-purpose language model designed to provide answers and insights across a wide array of topics not specifically categorized under other specialized models. This tool is specifically useful for users seeking information on miscellaneous and diverse topics that do not fall into the standard academic or professional categories such as physics, chemistry, biology, computer science, math, electrical engineering, history, philosophy, law, politics, culture, economics, geography, psychology, business, or health.

    Parameters:
    - query (str): A general prompt encompassing any topic of interest outside the specified categories. Speak in a broad and inclusive manner.

    Returns:
    - str: Comprehensive explanations or information pertaining to the general query, ens

In [25]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import time
torch.random.manual_seed(0)

question = "Write an email to my friend arik@gmail.com about I couldn't attend his wedding."

inputs = f"<|system|>You are a router. Below is the query from the users, please call the correct function and generate the parameters to call the function.<|end|><|user|>{question}<|end|><|assistant|>"

print('\n============= Below is the response ==============\n')

# You should consider to use early stopping with <nexa_end> token to accelerate
input_ids = tokenizer(inputs, return_tensors="pt")['input_ids'].to(model.device)

generated_token_ids = []
start = time.time()

# set a large enough number here to avoid insufficient length
for i in range(200):
    next_token = model(input_ids).logits[:, -1].argmax(-1)
    generated_token_ids.append(next_token.item())

    input_ids = torch.cat([input_ids, next_token.unsqueeze(1)], dim=-1)

    # 32041 is the token id of <nexa_end>
    if next_token.item() == 32041:
        break

print(tokenizer.decode(generated_token_ids))
end = time.time()
print(f'Elapsed time: {end - start:.2f}s')



============= Below is the response ==============

<nexa_16> ('Craft a heartfelt email to a friend explaining the absence from his wedding ceremony.')<nexa_end>
Elapsed time: 7.94s


In [34]:
def get_weather_forecast(location):
    """
    Provides a weather forecast for a specified location over a given number of days. Each day's forecast includes a brief description of the expected weather conditions.

    Parameters:
    - location (str): The location for which the weather forecast is desired. Can be a city name, ZIP code, or other location identifiers.

    Returns:
    - list[str]: A list of strings, each representing the weather forecast for one day. Each string includes the date and a brief description of the weather conditions. Formatted in 'YYYY-MM-DD: Description' format.
    """

def search_youtube_videos(query):
    """
    Searches YouTube for videos matching a query.

    Parameters:
    - query (str): Search query.

    Returns:
    - list[str]: A list of strings, each string includes video names and URLs.
    """
def send_email(recipient, title, content) ->str:
    """
    Sends an email to a specified recipient with a given title and content.

    Parameters:
    - recipient (str): The email address of the recipient.
    - title (str): The subject line of the email. This is a brief summary or title of the email's purpose or content.
    - content (str): The main body text of the email. It contains the primary message, information, or content that is intended to be communicated to the recipient.

    Returns:
    """

def get_trending_news(category=None, region='US', language='en',max_results=5):
    """
    Fetches trending news articles based on category, region, and language.
    Parameters:
    - category (str, optional): News category to filter by, by default use
    None for all categories. Optional to provide.
    - region (str, optional): ISO 3166-1 alpha-2 country code for regionspecific news, by default, uses ’US’. Optional to provide.
    - language (str, optional): ISO 639-1 language code for article
    language, by default uses ’en’. Optional to provide.
    - max_results (int, optional): Maximum number of articles to return, by
    default, uses 5. Optional to provide.
    Returns:
    - list[str]: A list of strings, each representing an article. Each
    string contains the article’s heading and URL.
    """

def make_phone_call(phone_number):
    """
    Initiates a phone call to the given phone number. It can handle both international and domestic numbers.

    Parameters:
    - phone_number (str): phone number of the contact. The phone number should be provided in a standard format, preferably in E.164 format (e.g., +12345678900 for an international format).

    Returns:
    """

In [35]:
def generate_octopus(prompt):
    fn = """{"name": "function_name", "arguments": {"arg_1": "value_1", "arg_2": value_2, ...}}"""
    prompt = f"""system
You are a helpful assistant with access to the following functions:
{make_phone_call}
Sends an email to a specified recipient with a given title and content
<nexa_i>user{prompt}<nexa_end>
<nexa_3>assistant<nexa_end>\n
"""

    tokens = tokenizer(prompt, return_tensors="pt").to(model.device)
    input_size = tokens.input_ids
    with torch.inference_mode():
        generated_tokens = model.generate(**tokens, use_cache=False, do_sample=True, temperature=0.5, top_p=0.8, top_k=50, max_new_tokens=4096)
    return tokenizer.decode(generated_tokens[0],skip_special_tokens=False)
    # return tokenizer.decode(generated_tokens.squeeze()[input_size:], skip_special_tokens=True)

In [36]:
prompt="I am in emergency. Make a phone call"
# prompt1="Recipient is sawradip0. The title will be Extremely sorry for not comming to the function. The content is I couldn't attend the function due to sickness."
completion = generate_octopus(prompt)
# functions = extract_function_calls(completion)
# print(functions)

In [37]:
from pprint import pprint
pprint(completion)

('<s> system\n'
 'You are a helpful assistant with access to the following functions:\n'
 '<function make_phone_call at 0x7be11cc08700>\n'
 'Sends an email to a specified recipient with a given title and content\n'
 '<nexa_i>userI am in emergency. Make a phone call<nexa_end> \n'
 '<nexa_3> assistant<nexa_end> \n'
 '\n'
 'Function description: \n'
 'def general_gpt(query):\n'
 '    """\n'
 '    A general-purpose language model designed to provide answers and '
 'insights across a wide array of topics not specifically categorized under '
 'other specialized models. This tool is specifically useful for users seeking '
 'information on miscellaneous and diverse topics that do not fall into the '
 'standard academic or professional categories such as physics, chemistry, '
 'biology, computer science, math, electrical engineering, history, '
 'philosophy, law, politics, culture, economics, geography, psychology, '
 'business, or health.\n'
 '\n'
 '    Parameters:\n'
 '    - query (str): A ge

In [38]:
pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.6/314.6 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 8.5 MB/s e

**GRADIO CHECKING**

In [39]:
import gradio as gr
def generate_response(input_text):
    nexa_query=f"""Below is the query from the users, please call the correct function and generate parameters
    to call the function. \n\nQuery: {input_text} \n\nResponse:"""

    input_ids=tokenizer(nexa_query, return_tensors="pt").to(model.device)
    input_length=input_ids["input_ids"].shape[1]
    outputs=model.generate(
        input_ids=input_ids["input_ids"],
        max_length=1024,
        do_sample=True
    )
    generated_sequence=outputs[:, input_length:].tolist()
    res=tokenizer.decode(generated_sequence[0])
    return res

iface = gr.Interface(fn=generate_response, inputs="text", outputs="text", title="Octopus v4 Function Generator", description="Enter the input text and get the generated email response.")
iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://10097a1a9eeb059f01.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
